# AI Project Sub3

In [1]:
from konlpy.tag import Kkma
import pandas as pd
import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np
from configs import DEFINES

ModuleNotFoundError: No module named 'configs'

# 데이터셋에 데이터 추가하기
## http://www.aihub.or.kr/ 에서 데이터를 가져왔다.
    - 엑셀 파일의 이름은 변경해야 한다.
    - 엑셀의 내용 중 필요한 내용만 따로 빼서 csv파일로 만들었다.
    - 길이가 너무 긴 질문이나 답은 제외 시켰다.

In [ ]:
import openpyxl
import csv

In [ ]:
# 엑셀 파일 열기
names = ['academy.xlsx', 'accommodation.xlsx', 'caffe.xlsx', 'cloth.xlsx', 'estate.xlsx', 'food.xlsx', 'play.xlsx', 'service.xlsx', 'store.xlsx']

for name in names:
    wb = openpyxl.load_workbook(name)
    # sheet 열기
    sheet = wb['Sheet1']
    # 마지막 row 셀 찾기
    max_row = sheet.max_row
    f = open('ChatBotData.csv', 'a', encoding='utf-8', newline='')
    wr = csv.writer(f)
    for r in range(2, max_row):
        # 질문과 답이 모두 있는것만 csv파일에 넣기
        if sheet.cell(row=r, column=13).value and sheet.cell(row=r+1, column=16).value:
            q = sheet.cell(row=r, column=13).value
            a = sheet.cell(row=r+1, column=16).value
            if len(q) < 25 and len(a) < 25:
                wr.writerow([q, a, 0])
    f.close()
    wb.close()


names = ['vehicle.xlsx', 'traffic.xlsx', 'waterworks.xlsx', 'passport.xlsx']
for name in names:
    wb = openpyxl.load_workbook(name)
    sheet = wb['Sheet']
    max_row = sheet.max_row
    f = open('ChatBotData.csv', 'a', encoding='utf-8', newline='')
    wr = csv.writer(f)
    for r in range(2, max_row):
        q = sheet.cell(row=r, column=5).value
        a = sheet.cell(row=r, column=6).value
        if q and a:
            if len(q) < 25 and len(a) < 25:
                wr.writerow([q, a, 0])
    f.close()
    wb.close()

# 데이터 분석
## wordcloud 만들기

In [ ]:
# 데이터 분석

import pandas as pd
from konlpy.tag import Okt
from tqdm import tqdm
from wordcloud import WordCloud
import matplotlib.pyplot as plt

data = pd.read_csv('./ChatBotData.csv', encoding='utf-8')

q_sentences = list(data['Q'])
a_sentences = list(data['A'])

# 토크나이즈
q_token_sentences = []
a_token_sentences = []

okt = Okt() # 객체 생성

for s in tqdm(q_sentences):
    for token, tag in okt.pos(s.replace(' ', '')):
        if tag == 'Noun':
            q_token_sentences.append(token)
            
for s in tqdm(a_sentences):
    for token, tag in okt.pos(s.replace(' ', '')):
        if tag == 'Noun':
            a_token_sentences.append(token)

# 하나의 문자열로 만들기
q_token_sentences = ' '.join(q_token_sentences)
a_token_sentences = ' '.join(a_token_sentences)

# # jupyter notebook 한글 폰트 
# from matplotlib import font_manager, rc
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# rc('font', family=font_name)

# 질문관련 wordcloud
q_wordcloud = WordCloud('C:/Windows/Fonts/NanumGothic.ttf').generate(q_token_sentences)

plt.imshow(q_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# 대답관련 wordcloud
a_wordcloud = WordCloud('C:/Windows/Fonts/NanumGothic.ttf').generate(a_token_sentences)

plt.imshow(a_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


# Project
## data.py

In [ ]:
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]

# Req 1-1-1. 데이터를 읽고 트레이닝 셋과 테스트 셋으로 분리

In [ ]:
# Req 1-1-1. 데이터를 읽고 트레이닝 셋과 테스트 셋으로 분리
def load_data(file):
    datas = pd.read_csv(DEFINES.data_path, header=0)
    q, a = list(datas['Q']), list(datas['A'])
    
    train_q, test_q, train_a, test_a = train_test_split(q, a, test_size=0.35, random_state=321)
    return train_q, train_a, test_q, test_a

In [ ]:
# train_q, train_a, test_q, test_a = load_data("./data_in/ChatBotData.csv")

# Req 1-1-2. 텍스트 데이터에 정규화를 사용하여 ([~.,!?\"':;)(]) 제거

In [1]:
def prepro_noise_canceling(sentence):
    RE_FILTER = re.compile("[.,!?\"':;~()]")
    sentence = re.sub(RE_FILTER, "", sentence)
    return sentence

# Req 1-1-3. 텍스트 데이터에 토크나이징

### 알고리즘 순서
    - 텍스트 데이터 prepro_noise_canceling() 함수 처리
    - 띄어쓰기 단위로 나누기
    - 띄어진 단어들로 벡터 형성

In [ ]:
def tokenizing_data(data):
    data = prepro_noise_canceling(data)
    data_splited = list(data.split())
    
    return data_splited

# Req 1-2-1. 토큰화된 트레이닝 데이터를 인코더에 활용할 수 있도록 전 처리

### 다음 내용들이 적용되어야 함
    - 텍스트 데이터 prepro_noise_canceling()함수 처리
    - 문장을 토큰 단위로 나누기
    - dictionary를 활용하여 토큰 인덱스화
    - dictionary에 없는 토큰의 경우 UNK 값으로 대체
    - 기준 문장 길이 보다 크게 된다면 뒤의 토큰 자르기
    - 기준 문장 길이에 맞게 남은 공간이 있다면 pedding 하기
    - 문장 인덱스와 문장 길이 계산

In [1]:
# Req 1-2-1. 토큰화된 트레이닝 데이터를 인코더에 활용할 수 있도록 전 처리
def enc_processing(value, dictionary):
    
    # 인덱스 정보를 저장할 배열 초기화
    seq_input_index = []
    # 문장의 길이를 저장할 배열 초기화
    seq_len = []
    # 노이즈 캔슬
    value = prepro_noise_canceling(value)
    
    for seq in value:
        # 하나의 seq에 index를 저장할 배열 초기화
        seq_index =[]

        for word in seq.split():
            if dictionary.get(word) is not None:
                # seq_index에 dictionary 안의 인덱스를 extend 한다
                seq_index.append(dictionary[word])
            else:
                # dictionary에 존재 하지 않는 다면 UNK 값을 extend 한다
                seq_index.append(dictionary[UNK])

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 제거
        if len(seq_index) > DEFINES.max_sequence_length:
            seq_index = seq_index[:DEFINES.max_sequence_length]

        # seq의 길이를 저장
        seq_len.append(len(seq_index))

        # DEFINES.max_sequence_length 길이보다 작은 경우 PAD 값을 추가 (padding)
        seq_index += (DEFINES.max_sequence_length - len(seq_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값은 seq_input_index에 추가
        seq_input_index.append(seq_index)

    return np.asarray(seq_input_index)

# Req 1-2-2. 디코더에 필요한 데이터 전 처리 

### 다음 내용들이 적용되어야 함
    - 텍스트 데이터 prepro_noise_canceling()함수 처리
    - 문장을 토큰 단위로 나누기
    - dictionary를 활용하여 토큰 인덱스화
    - 첫 인덱스에 STD 추가
    - dictionary에 없는 토큰의 경우 UNK 값으로 대체
    - 기준 문장 길이 보다 크게 된다면 뒤의 토큰 자르기
    - 기준 문장 길이에 맞게 남은 공간이 있다면 pedding 하기
    - 문장 인덱스와 문장 길이 계산

In [1]:

def dec_input_processing(value, dictionary):
    
    # 인덱스 정보를 저장할 배열 초기화
    seq_input_index = []
    # 문장의 길이를 저장할 배열 초기화
    seq_len = []
    # 노이즈 캔슬
    value = prepro_noise_canceling(value)

    for seq in value:
        # 하나의 seq에 index를 저장할 배열 초기화
        seq_index =[]

        for word in seq.split():
            # 디코딩 입력의 처음에는 START가 와야 하므로 STD 값 추가
            seq_index.append(dictionary[STD])

            if dictionary.get(word) is not None:
                # seq_index에 dictionary 안의 인덱스를 extend 한다
                seq_index.append(dictionary[word])
            else:
                # dictionary에 존재 하지 않는 다면 seq_index에 UNK 값을 extend 한다
                seq_index.append(dictionary[UNK])

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 제거
        if len(seq_index) > DEFINES.max_sequence_length:
            seq_index = seq_index[:DEFINES.max_sequence_length]

        # seq의 길이를 저장
        seq_len.append(len(seq_index))

        # DEFINES.max_sequence_length 길이보다 작은 경우 PAD 값을 추가 (padding)
        seq_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값은 seq_input_index에 추가
        seq_input_index.append(seq_index)
    
    return np.asarray(seq_input_index)

# Req 1-2-3. 디코더에 필요한 데이터 전 처리 

In [1]:
# Req 1-2-3. 디코더에 필요한 데이터 전 처리 
def dec_target_processing(value, dictionary):
    
    # 인덱스 정보를 저장할 배열 초기화
    seq_input_index = []
    # 문장의 길이를 저장할 배열 초기화
    seq_len = []
    # 노이즈 캔슬
    value = prepro_noise_canceling(value)

    for seq in value:

        # 하나의 seq에 index를 저장할 배열 초기화
        seq_index =[]

        seq_index = [dictionary[word] for word in seq.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 제거
        # END 토큰을 추가 (DEFINES.max_sequence_length 길이를 맞춰서 추가)
        
        seq_index = seq_index[:DEFINES.max_sequence_length-1] + [dictionary[END]]

        # seq의 길이를 저장
        seq_len.append(len(seq_index))

        # DEFINES.max_sequence_length 길이보다 작은 경우 PAD 값을 추가 (padding)
        seq_index += ( DEFINES.max_sequence_length - len(seq_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값은 seq_input_index에 추가
        seq_input_index.append(seq_index)
   
    return np.asarray(seq_input_index)

In [1]:
# input과 output dictionary를 만드는 함수
def in_out_dict(input, output, target):
    features = {"input": input, "output": output}
    return features, target


# 학습에 들어가 배치 데이터를 만드는 함수
def train_input_fn(train_input_enc, train_input_dec, train_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은
    # 각각 한 문장으로 자른다고 보면 된다.
    # train_input_enc, train_output_dec, train_target_dec
    # 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((train_input_enc, train_input_dec, train_target_dec))
    # 전체 데이터를 썩는다.
    dataset = dataset.shuffle(buffer_size=len(train_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "train batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을
    # 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size, drop_remainder=True)
    # 데이터 각 요소에 대해서 in_out_dict 함수를
    # 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(in_out_dict)
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면
    # 아무 인자도 없다면 무한으로 이터레이터 된다.
    dataset = dataset.repeat()
    # make_one_shot_iterator를 통해 이터레이터를
    # 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 텐서
    # 개체를 넘겨준다.
    return iterator.get_next()


# 평가에 들어가 배치 데이터를 만드는 함수
def eval_input_fn(eval_input_enc, eval_input_dec, eval_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은
    # 각각 한 문장으로 자른다고 보면 된다.
    # eval_input_enc, eval_input_dec, eval_target_dec
    # 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((eval_input_enc, eval_input_dec, eval_target_dec))
    # 전체 데이터를 섞는다.
    dataset = dataset.shuffle(buffer_size=len(eval_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "eval batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을
    # 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size, drop_remainder=True)
    # 데이터 각 요소에 대해서 in_out_dict 함수를
    # 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(in_out_dict)
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면
    # 아무 인자도 없다면 무한으로 이터레이터 된다.
    # 평가이므로 1회만 동작 시킨다.
    dataset = dataset.repeat(1)
    # make_one_shot_iterator를 통해
    # 이터레이터를 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의
    # 텐서 개체를 넘겨준다.
    return iterator.get_next()

# Req 1-3-1. 단어 사전 파일 vocabularyData.voc를 생성하고 단어와 인덱스 관계를 출력

In [2]:
# Req 1-3-1. 단어 사전 파일 vocabularyData.voc를 생성하고 단어와 인덱스 관계를 출력
def load_voc():
    # 사전을 담을 배열 준비한다.
    voc_list = []
    # 사전을 구성한 후 파일로 저장 진행한다.
    # 그 파일의 존재 유무를 확인한다.
    if (not (os.path.exists(DEFINES.vocabulary_path))):
        # 이미 생성된 사전 파일이 존재하지 않으므로
        # 데이터를 가지고 만들어야 한다.
        # 그래서 데이터가 존재 하면 사전을 만들기 위해서
        # 데이터 파일의 존재 유무를 확인한다.
        
        data_df = pd.read_csv("./data_in/ChatBotData.csv")
        # 판다스의 데이터 프레임을 통해
        # 질문과 답에 대한 열을 가져 온다.
        question, answer = data_df["Q"], data_df["A"]
        question = prepro_like_morphlized(question)

        data = []
        # 질문과 답변을 extend을
        # 통해서 구조가 없는 배열로 만든다.
        data.extend(question)
        data.extend(answer)

        # data를 토크나이즈하여 words에 저장한다. 
        words = tokenizing_data(data)
        
        # 중복되는 단어(토큰)를 제거
        words = list(set(words))
        
        # 데이터 없는 내용중에 MARKER 추가
        words[:0] = MARKER
        
        # 사전 파일을 생성 
        # DEFINES.vocabulary_path에 words안에 저장된 가 단어(토큰)들을 한줄 씩 저장
        with open(DEFINES.vocabulary_path, 'w', encoding='utf-8') as voc_file:
            for word in words:
                voc_file.write(word + "\n")
           
    # 사전 파일에서 단어(토큰)을 가져와 voc_list에 저장
    with open(DEFINES.vocabulary_path, 'r', encoding='utf-8') as voc_file:
        for line in voc_file:
            voc_list.append(line.strip())

    # make() 함수를 사용하여 dictionary 형태의 char2idx, idx2char 저장
    char2idx, idx2char = make_voc(voc_list)

    return char2idx, idx2char, len(char2idx)

IndentationError: expected an indented block (<ipython-input-2-fdf24aa7b622>, line 36)

# Req 1-3-2. 사전 리스트를 받아 인덱스와 토큰의 dictionary를 생성


In [ ]:
def make_voc(voc_list):
    char2idx = {}
    idx2char = {}
    for idx, char in enumerate(voc_list):
        char2idx[char] = idx
        idx2char[idx] = char
    
    return char2idx, idx2char

# Req 1-3-3. 예측용 단어 인덱스를 문장으로 변환

In [ ]:
# Req 1-3-3. 예측용 단어 인덱스를 문장으로 변환
def pred_next_string(value, dictionary):
    string_list = []
    
    for v in value:
        for index in v["indexs"]:
            string_list.append(dictionary[index])
    
    answer = ""
    
    for word in string_list:
        if word not in PAD and word not in END:
            answer += word
            answer += " "
            
    return answer, True

# Flask RESTful API

### flask restful 설치

In [2]:
pip install flask-restful

^C
Note: you may need to restart the kernel to use updated packages.


## db_init.py

In [1]:
import sqlite3
import os

if not os.path.isdir('db'):
    os.mkdir('db')

conn = sqlite3.connect('./db/app.db')

c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS thinkB (
        question TEXT,
        answer TEXT,
        create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )''')

c.execute('''Select * FROM thinkB;''')

data = c.fetchall() #list
conn.commit()
conn.close()

## app.py

In [ ]:
from flask import Flask, render_template
from flask_restful import Resource, Api, reqparse
import sqlite3

app = Flask(__name__)
api = Api(app)

def insertDB(q, a):
    conn = sqlite3.connect('./db/app.db')
    cur = conn.cursor()
    cur.execute("insert into thinkB (question, answer) values (:question, :answer)", {'question': q, 'answer': a})
    conn.commit()
    conn.close()

parser = reqparse.RequestParser()

class Chat(Resource):
    def post(self):
        parser.add_argument('question', type=str)
        parser.add_argument('answer', type=str)

        args = parser.parse_args()

        _question = args['question']
        _answer = args['answer']

        insertDB(_question, _answer)
        return {'question': args['question'], 'answer': args['answer']}

api.add_resource(Chat, '/chat')

if __name__ == "__main__":
    app.run(debug=True)

# Transformer Model

# Chat class 
### flask restful - post

In [ ]:
class Chat(Resource):
    def post(self):
        try:
            parser = reqparse.RequestParser()
            parser.add_argument('question', type=str)

            args = parser.parse_args()

            _question = args['question']
            _answer = predict(_question)

            datas = {
                'flag': True,
                'answer': _answer
            }
            
            return datas

        except Exception as e:
            datas = {
                'flag': False,
                'answer': str(e)
            }
            return datas
    

api.add_resource(Declaration, '/Declaration')
api.add_resource(Chat, '/chat')